# Studio: Working with Databases in Python

For today's studio, we will be using the [TV Shows dataset](https://www.kaggle.com/ruchi798/tv-shows-on-netflix-prime-video-hulu-and-disney) from Kaggle. We have already downloaded the CSV for you.

You will use the watchlist you created to answer these questions:

1. **Which streaming services contain the shows you want to watch next?**
2. **Which streaming service is the best value based on the shows you want to watch?**

As you complete the different tasks in the studio, you may choose between using Pandas or SQL. 

**Remember**: we learned in our prep work that one is oftentimes more efficient at certain tasks than the other, so choose wisely!

## My Watchlist

If you would like, please use this space to make note of your watchlist by editing the text cell. You will need 10 shows overall.

1. Dawn of the Croods
2. Avatar: The Last Airbender
3. 101 Dalmatians
4. The Lion Guard
5. OMG!
6. Honolulu P.D.
7. New Amsterdam
8. Cleopatra
9. Kulipari: An Army of Frogs 
10. The Practice

## Database Setup

Import the necessary libraries and create a dataframe from the provided CSV. 

Print the info out for the dataframe. 

After that, you may drop the column called `Unnamed: 0` and rename any columns with spaces or unusual characters in the names such as `"Disney+"`. 

Print out the info for the dataframe again to ensure your changes were made.

In [63]:
# Code here
import pandas as pd
import sqlite3

#  create the dataframe from the csv file 
df = pd.read_csv("tv_shows.csv")

print(df)

      Unnamed: 0    ID                                  Title  Year  Age  \
0              0     1                           Breaking Bad  2008  18+   
1              1     2                        Stranger Things  2016  16+   
2              2     3                        Attack on Titan  2013  18+   
3              3     4                       Better Call Saul  2015  18+   
4              4     5                                   Dark  2017  16+   
...          ...   ...                                    ...   ...  ...   
5363        5363  5713                       Paradise Islands  2017  NaN   
5364        5364  5714                         Mexico Untamed  2018  NaN   
5365        5365  5715                  Wild Centeral America  2020  NaN   
5366        5366  5716                            Wild Russia  2018  NaN   
5367        5367  5717  Fearless Adventures with Jack Randall  2018  NaN   

        IMDb Rotten Tomatoes  Netflix  Hulu  Prime Video  Disney+  Type  
0     9.4/10 

In [64]:
# list the columns of the tv_shows dataframe
df.columns
# 'Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
#        'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type'],
#       dtype='object'

# drop the 'Unnamed: 0' column
df = df.drop(columns=['Unnamed: 0'], axis=1)

# Rename multiple columns
df.rename(columns={'Rotten Tomatoes': 'Rotten_Tomatoes', 
                   'Prime Video': 'Prime_Video',
                   'Disney+': 'Disney_Plus'}, inplace=True)

print(df)

        ID                                  Title  Year  Age    IMDb  \
0        1                           Breaking Bad  2008  18+  9.4/10   
1        2                        Stranger Things  2016  16+  8.7/10   
2        3                        Attack on Titan  2013  18+  9.0/10   
3        4                       Better Call Saul  2015  18+  8.8/10   
4        5                                   Dark  2017  16+  8.8/10   
...    ...                                    ...   ...  ...     ...   
5363  5713                       Paradise Islands  2017  NaN     NaN   
5364  5714                         Mexico Untamed  2018  NaN     NaN   
5365  5715                  Wild Centeral America  2020  NaN     NaN   
5366  5716                            Wild Russia  2018  NaN     NaN   
5367  5717  Fearless Adventures with Jack Randall  2018  NaN     NaN   

     Rotten_Tomatoes  Netflix  Hulu  Prime_Video  Disney_Plus  Type  
0            100/100        1     0            0            0    

With your dataframe at the ready, create a new database called `tv.db`. 

Add a new table to your database called `shows` using the data in the dataframe. 

In [66]:
# Code here
# connect to a SQLite database. If 'tv.db' database file doesnt exist a new one is created
connection = sqlite3.connect('tv.db')

# create a cursor to execute commands using the connection
cursor = connection.cursor()

# Method 1
# CREATE NEW TABLE IN DATABASE FILE USING PYTHON DATAFRAME
# # Write the DataFrame into database as new table 'shows', if exsists replace it
# # and don't add an index column
# df.to_sql('shows_table', connection, if_exists="replace", index=False)

# Method 2
# CREATE NEW TABLE IN DATABSE FILE USING PYTHON SQLITE3
# execute command to create a new table shows_table with the df
# dataframe data
connection.execute('''
        CREATE TABLE IF NOT EXISTS shows_table as
                   select * from df
                   ''')

With your new table and database set up, print out the top 20 records in the `shows` table.

In [67]:
# Code Here
# read data from newly created table
shows_table_df = pd.read_sql_query('Select * from shows_table', connection)
shows_table_df.head()

,index,ID,Title,Year,Age,IMDb,Rotten_Tomatoes,Netflix,Hulu,Prime_Video,Disney_Plus,Type
0,0,1,Breaking Bad,2008,18+,9.4/10,100/100,1,0,0,0,1
1,1,2,Stranger Things,2016,16+,8.7/10,96/100,1,0,0,0,1
2,2,3,Attack on Titan,2013,18+,9.0/10,95/100,1,1,0,0,1
3,3,4,Better Call Saul,2015,18+,8.8/10,94/100,1,0,0,0,1
4,4,5,Dark,2017,16+,8.8/10,93/100,1,0,0,0,1


Now, create a new table called `watchlist` that has three fields:
1. id -> data type of `INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT`
2. title -> data type of `TEXT`
3. importance_rank -> data type of `INTEGER`

For the `importance_rank` field, rank each of your watchlist shows based on how much you want to see them, `10` being the most important and `1` being the least important.

Then, insert each of the items from your watchlist into the new `watchlist` table, using the `executemany` method from our exercises.

Finally, select all the records from the `watchlist` table and print them out to the console.

In [78]:
# Create table

# cursor.execute("Drop table watchlist")
cursor.execute("CREATE TABLE watchlist (id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, title TEXT, importance_rank INTEGER)")


In [79]:
# INSERT data into the new table

sql = 'INSERT INTO watchlist (title, importance_rank) values(?,?)'
data = [ ("Avatar: The Last Airbender", 6),
          ("OMG!", 1),
          ("Honolulu P.D.", 8),
          ("The Lion Guard",2),
          ("The Practice",9),
          ("Kulipari: An Army of Frogs ",3),
          ("101 Dalmatians",4),
          ("Dawn of the Croods", 7),
          ("New Amsterdam", 10),
          ("Cleopatra", 5)
]
with connection:
    # the sqlite3 `executemany` method to run a sql statement for each element within a list
    connection.executemany(sql, data)

In [80]:
# read data

with connection:
    for row in cursor.execute("SELECT * FROM watchlist"):
        print(row)

(1, 'Avatar: The Last Airbender', 6)
(2, 'OMG!', 1)
(3, 'Honolulu P.D.', 8)
(4, 'The Lion Guard', 2)
(5, 'The Practice', 9)
(6, 'Kulipari: An Army of Frogs ', 3)
(7, '101 Dalmatians', 4)
(8, 'Dawn of the Croods', 7)
(9, 'New Amsterdam', 10)
(10, 'Cleopatra', 5)


## Working with the Data

Using Pandas or SQL, find the answer to these 2 questions:
1. How many of the total shows (full csv list) are on each streaming service?
2. What percentage of these total shows is available on each streaming service?

**Hint**:

Use the pandas `query` method to filter the data, and then the Python `len` method to find it's length. [Relevant Link](https://www.geeksforgeeks.org/ways-to-filter-pandas-dataframe-by-column-values/)

In [90]:
# Code here
# list of the streaming services in the dataset
service_columns = ['Netflix', 'Hulu', 'Prime_Video', 'Disney_Plus']

# Initialize a dictionary to store counts
service_counts = {}

# Total shows on each streaming service
for service in service_columns:
    service_counts[service] = df[service].sum() 

# Convert to a Series for easier manipulation
service_counts = pd.Series(service_counts)

print("Total shows on each streaming service:")
print(service_counts)


# Calculate total number of shows
total_shows = len(df)
print("Total number of shows:", total_shows)

# Calculate percentage of shows on each service
service_percentage = (service_counts / total_shows) * 100

print("\nPercentage of total shows on each streaming service:")
print(service_percentage)

Total shows on each streaming service:
Netflix        1971
Hulu           1621
Prime_Video    1831
Disney_Plus     351
dtype: int64
Total number of shows: 5368

Percentage of total shows on each streaming service:
Netflix        36.717586
Hulu           30.197466
Prime_Video    34.109538
Disney_Plus     6.538748
dtype: float64



Now join your `watchlist` data to the `shows` data using pandas or SQL. Verify that you joined the data correctly.

Using this related dataset, come up with analytic code that answers these questions:
1. The number of watchlist shows each streaming service has
2. The percentage of your overall watchlist each streaming service has


In [91]:
# Code here



## Results

Now that you have done your analysis, make note of the answers to the following questions by editing the text cell:

1. Was every show on your watchlist in the Kaggle dataset? Do you have any ideas as to why a show might not have been present?

- *your_answer*

2. Did you include a show or shows in your watchlist that is exclusive to one of the platforms? How might that have impacted your analysis?

- *your_answer*

3. Which streaming service(s) offered the most shows on your watchlist? Which streaming service(s) offered the least?

- *your_answer*

4. Based on the shows you want to watch and the results of your analysis, is there a streaming service you think would be a good fit for you?

- *your_answer*

# Bonus Mission

We didn't end up using that `importance_rank` field, did we?

Well, that was intentional! 

Your bonus mission is to come up with analysis that uses that field to determine, based on watchlist show importance_rank and number of watchlist shows available on a service, which platform you should subscribe to.

In [ ]:
# Code Here